In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, DateType
from pyspark.sql import Window as W
import pyspark.sql.functions as F

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("SQL Interview Questions") \
    .getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/07 14:42:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [78]:
# Wordcount problem
from pyspark.sql import functions as F 
df = spark.createDataFrame(["Hi i am durai. i am writing a word count program using spark df"],"string")
df.select(F.explode(F.split("value"," ")).alias("word")).groupBy("word").agg(F.count(F.lit(1))).show(1000,False)

#RDD
rdd = df.rdd
words_rdd = rdd.flatMap(lambda line: line.value.split(" "))
word_pairs_rdd = words_rdd.map(lambda word: (word, 1))
word_counts_rdd = word_pairs_rdd.reduceByKey(lambda a, b: a + b)
word_counts_rdd.collect()

+-------+--------+
|word   |count(1)|
+-------+--------+
|Hi     |1       |
|a      |1       |
|count  |1       |
|am     |2       |
|program|1       |
|using  |1       |
|spark  |1       |
|df     |1       |
|writing|1       |
|i      |2       |
|durai. |1       |
|word   |1       |
+-------+--------+



[('Hi', 1),
 ('i', 2),
 ('using', 1),
 ('df', 1),
 ('a', 1),
 ('word', 1),
 ('program', 1),
 ('spark', 1),
 ('am', 2),
 ('writing', 1),
 ('count', 1),
 ('durai.', 1)]

In [2]:
# Q1: Different delimiter 
# 
from pyspark.sql.functions import split, col
data = ["1,Alice\t30|New York"]

# Creating a DataFrame with a single column
df = spark.createDataFrame(data, "string")

# Custom logic to split the mixed delimiter row
split_col = split(df['value'], ',|\t|\|')

# Creating new columns for each split part
df = df.withColumn('id', split_col.getItem(0))\
       .withColumn('name', split_col.getItem(1))\
       .withColumn('age', split_col.getItem(2))\
       .withColumn('city', split_col.getItem(3))

# Selecting and showing the result
df.select('id', 'name', 'age', 'city').show()

+---+-----+---+--------+
| id| name|age|    city|
+---+-----+---+--------+
|  1|Alice| 30|New York|
+---+-----+---+--------+



In [15]:
# Moving avg
from pyspark.sql import Row
data = [Row(Date='2023-01-01', ProductID=100, QuantitySold=10),
        Row(Date='2023-01-02', ProductID=100, QuantitySold=15),
        Row(Date='2023-01-03', ProductID=100, QuantitySold=20),
        Row(Date='2023-01-04', ProductID=100, QuantitySold=25),
        Row(Date='2023-01-05', ProductID=100, QuantitySold=30),
        Row(Date='2023-01-06', ProductID=100, QuantitySold=35),
        Row(Date='2023-01-07', ProductID=100, QuantitySold=40),
        Row(Date='2023-01-08', ProductID=100, QuantitySold=45)]

# Create DataFrame
df_sales = spark.createDataFrame(data)
df_sales.show()
df_sales.createOrReplaceTempView("Sales")
spark.sql("select *, avg(QuantitySold) over(partition by ProductID order by Date rows between 6 preceding and current row) as avg from Sales").show()

+----------+---------+------------+
|      Date|ProductID|QuantitySold|
+----------+---------+------------+
|2023-01-01|      100|          10|
|2023-01-02|      100|          15|
|2023-01-03|      100|          20|
|2023-01-04|      100|          25|
|2023-01-05|      100|          30|
|2023-01-06|      100|          35|
|2023-01-07|      100|          40|
|2023-01-08|      100|          45|
+----------+---------+------------+

+----------+---------+------------+----+
|      Date|ProductID|QuantitySold| avg|
+----------+---------+------------+----+
|2023-01-01|      100|          10|10.0|
|2023-01-02|      100|          15|12.5|
|2023-01-03|      100|          20|15.0|
|2023-01-04|      100|          25|17.5|
|2023-01-05|      100|          30|20.0|
|2023-01-06|      100|          35|22.5|
|2023-01-07|      100|          40|25.0|
|2023-01-08|      100|          45|30.0|
+----------+---------+------------+----+



In [37]:
from pyspark.sql.types import StructType, StructField, StringType

# Define schema
schema = StructType([
    StructField("event_date", StringType(), True),
    StructField("event_status", StringType(), True)
])

# Create data
data = [
    ("01-06-2020", "Won"),
    ("02-06-2020", "Won"),
    ("03-06-2020", "Won"),
    ("04-06-2020", "Lost"),
    ("05-06-2020", "Lost"),
    ("06-06-2020", "Lost"),
    ("07-06-2020", "Won"),
    ("08-06-2020", "Lost")
]

# Create DataFrame
df = spark.createDataFrame(data, schema)
df.createOrReplaceTempView("df")
# Show DataFrame
df.show()

+----------+------------+
|event_date|event_status|
+----------+------------+
|01-06-2020|         Won|
|02-06-2020|         Won|
|03-06-2020|         Won|
|04-06-2020|        Lost|
|05-06-2020|        Lost|
|06-06-2020|        Lost|
|07-06-2020|         Won|
|08-06-2020|        Lost|
+----------+------------+



In [40]:

spark.sql("""
with df1 as (
select *, if(lag(event_status) over(order by event_date) != event_status, 1, 0)  as nxt from df
) ,
df2 as (
    select *, sum(nxt) over(order by event_date) as sum from df1
) 
select distinct 
event_status,
min(event_date) over(partition by sum) as min_dt,
max(event_date) over(partition by sum) as max_dt
from df2
""").show()

24/09/06 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 17:43:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/06 1

+------------+----------+----------+
|event_status|    min_dt|    max_dt|
+------------+----------+----------+
|         Won|01-06-2020|03-06-2020|
|        Lost|04-06-2020|06-06-2020|
|         Won|07-06-2020|07-06-2020|
|        Lost|08-06-2020|08-06-2020|
+------------+----------+----------+



In [42]:
# Exchange Seats of Students

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("student", StringType(), True)
])

data = [
    (1, "Alice"),
    (2, "Bob"),
    (3, "Charlie"),
    (4, "David"),
    (5, "Eve")
]

df = spark.createDataFrame(data, schema)
df.show()
df.createOrReplaceTempView("df")

+---+-------+
| id|student|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
|  4|  David|
|  5|    Eve|
+---+-------+



In [50]:

spark.sql("""
select id, 
nvl(if (id %2 = 0, lag(student) over(partition by 1 order by id) ,lead(student) over(partition by 1 order by id)),student) as res from df
""").show()

+---+-------+
| id|    res|
+---+-------+
|  1|    Bob|
|  2|  Alice|
|  3|  David|
|  4|Charlie|
|  5|    Eve|
+---+-------+



In [51]:
# Number of Calls and Total Duration

data = [
    (10, 20, 58),
    (20, 10, 12),
    (10, 30, 20),
    (30, 40, 100),
    (30, 40, 200),
    (30, 40, 200),
    (40, 30, 500)
]

# Define schema for DataFrame
columns = ["from_id", "to_id", "duration"]

# Create DataFrame
df = spark.createDataFrame(data, schema=columns)

df.show()
df.createOrReplaceTempView("df")

+-------+-----+--------+
|from_id|to_id|duration|
+-------+-----+--------+
|     10|   20|      58|
|     20|   10|      12|
|     10|   30|      20|
|     30|   40|     100|
|     30|   40|     200|
|     30|   40|     200|
|     40|   30|     500|
+-------+-----+--------+



In [55]:
spark.sql("""
select 
if(from_id > to_id, to_id,from_id) from_id,
if(from_id > to_id, from_id,to_id) to_id,
count(1),
sum(duration)
from 
df
group by 1,2
""").show()

+-------+-----+--------+-------------+
|from_id|to_id|count(1)|sum(duration)|
+-------+-----+--------+-------------+
|     10|   20|       2|           70|
|     10|   30|       1|           20|
|     30|   40|       4|         1000|
+-------+-----+--------+-------------+



In [57]:
data1 = [
    (1, "Nadal"),
    (2, "Federer"),
    (3, "Novak")
]
col1 = ["player_id", "player_name"]

data2 = [
    (2017, 2, 1, 1, 2),
    (2018, 3, 1, 3, 2) ,
    (2019, 3, 1, 1, 3)
]
col2 = ["year", "Wimbledon", "Fr_open", "US_open", "Au_open"]

df1 = spark.createDataFrame(data1, schema=col1)
df2 = spark.createDataFrame(data2, schema=col2)

df1.show()
df2.show()
df1.createOrReplaceTempView("df1")
df2.createOrReplaceTempView("df2")


+---------+-----------+
|player_id|player_name|
+---------+-----------+
|        1|      Nadal|
|        2|    Federer|
|        3|      Novak|
+---------+-----------+

+----+---------+-------+-------+-------+
|year|Wimbledon|Fr_open|US_open|Au_open|
+----+---------+-------+-------+-------+
|2017|        2|      1|      1|      2|
|2018|        3|      1|      3|      2|
|2019|        3|      1|      1|      3|
+----+---------+-------+-------+-------+



In [61]:
spark.sql("""
with awards as (select year, Wimbledon as player_id from df2 union all
select year, Fr_open from df2 union all
select year, US_open from df2 union all
select year, Au_open from df2)
select df1.player_id,player_name, count(*) from df1 left join awards on df1.player_id = awards.player_id group by 1,2
""").show()

+---------+-----------+--------+
|player_id|player_name|count(1)|
+---------+-----------+--------+
|        2|    Federer|       3|
|        1|      Nadal|       5|
|        3|      Novak|       4|
+---------+-----------+--------+



In [87]:
data = [
    ("A1", "2024-01-01", "PRESENT"),
    ("A1", "2024-01-02", "PRESENT"),
    ("A1", "2024-01-03", "PRESENT"),
    ("A1", "2024-01-04", "ABSENT"),
    ("A1", "2024-01-05", "PRESENT"),
    ("A1", "2024-01-06", "PRESENT"),
    ("A1", "2024-01-07", "ABSENT"),
    ("A1", "2024-01-08", "ABSENT"),
    ("A1", "2024-01-09", "ABSENT"),
    ("A1", "2024-01-10", "PRESENT"),
    ("A2", "2024-01-06", "PRESENT"),
    ("A2", "2024-01-07", "PRESENT"),
    ("A2", "2024-01-08", "ABSENT"),
    ("A2", "2024-01-09", "PRESENT"),
    ("A2", "2024-01-10", "ABSENT")
]

# Define schema for the DataFrame
columns = ["EMPLOYEE", "DATES", "STATUS"]

# Create DataFrame
df = spark.createDataFrame(data, schema=columns)
df.createOrReplaceTempView("emp")
df.show()

+--------+----------+-------+
|EMPLOYEE|     DATES| STATUS|
+--------+----------+-------+
|      A1|2024-01-01|PRESENT|
|      A1|2024-01-02|PRESENT|
|      A1|2024-01-03|PRESENT|
|      A1|2024-01-04| ABSENT|
|      A1|2024-01-05|PRESENT|
|      A1|2024-01-06|PRESENT|
|      A1|2024-01-07| ABSENT|
|      A1|2024-01-08| ABSENT|
|      A1|2024-01-09| ABSENT|
|      A1|2024-01-10|PRESENT|
|      A2|2024-01-06|PRESENT|
|      A2|2024-01-07|PRESENT|
|      A2|2024-01-08| ABSENT|
|      A2|2024-01-09|PRESENT|
|      A2|2024-01-10| ABSENT|
+--------+----------+-------+



In [88]:
spark.sql("""
with cte as (
select *, 
sum(if(lag(STATUS) over(partition by EMPLOYEE order by DATES) != STATUS, 1, 0)) over(partition by EMPLOYEE order by DATES) as chk 
from emp )
select distinct 
EMPLOYEE ,
min(DATES) over (partition by EMPLOYEE,chk) as frm,
max(DATES) over (partition by EMPLOYEE,chk) as to,
STATUS
from cte
""").show()

+--------+----------+----------+-------+
|EMPLOYEE|       frm|        to| STATUS|
+--------+----------+----------+-------+
|      A1|2024-01-01|2024-01-03|PRESENT|
|      A1|2024-01-04|2024-01-04| ABSENT|
|      A1|2024-01-05|2024-01-06|PRESENT|
|      A1|2024-01-07|2024-01-09| ABSENT|
|      A1|2024-01-10|2024-01-10|PRESENT|
|      A2|2024-01-06|2024-01-07|PRESENT|
|      A2|2024-01-08|2024-01-08| ABSENT|
|      A2|2024-01-09|2024-01-09|PRESENT|
|      A2|2024-01-10|2024-01-10| ABSENT|
+--------+----------+----------+-------+



In [103]:
spark.sql("""
with cte as 
		(select *, row_number() over(partition by employee order by employee, dates) as rn 
		from emp),
	cte_present as
		(select *, row_number() over(partition by employee order by employee, dates)
		, rn - row_number() over(partition by employee order by employee, dates) as flag
		from cte where status='PRESENT' ),
	cte_absent as
		(select *, row_number() over(partition by employee order by employee, dates)
		, rn - row_number() over(partition by employee order by employee, dates) as flag
		from cte where status='ABSENT' )
select employee 
, min(dates) over(partition by employee, flag) as from_date 
, max(dates) over(partition by employee, flag) as to_date 
, status						
from cte_present
union 
select  employee 
, min(dates) over(partition by employee, flag) as from_date 
, max(dates) over(partition by employee, flag) as to_date 
, status						
from cte_absent
order by employee, from_date
""").show()

+--------+----------+----------+-------+
|employee| from_date|   to_date| status|
+--------+----------+----------+-------+
|      A1|2024-01-01|2024-01-03|PRESENT|
|      A1|2024-01-04|2024-01-04| ABSENT|
|      A1|2024-01-05|2024-01-06|PRESENT|
|      A1|2024-01-07|2024-01-09| ABSENT|
|      A1|2024-01-10|2024-01-10|PRESENT|
|      A2|2024-01-06|2024-01-07|PRESENT|
|      A2|2024-01-08|2024-01-08| ABSENT|
|      A2|2024-01-09|2024-01-09|PRESENT|
|      A2|2024-01-10|2024-01-10| ABSENT|
+--------+----------+----------+-------+



In [114]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

lift_schema = StructType([
    StructField("ID", IntegerType(), True),
    StructField("CAPACITY_KG", IntegerType(), True)
])

lift_data = [(1, 300), (2, 350)]
df_lift = spark.createDataFrame(lift_data, schema=lift_schema)
df_lift.show()
df_lift.createOrReplaceTempView("lift")

passengers_schema = StructType([
    StructField("PASSENGER_NAME", StringType(), True),
    StructField("WEIGHT_KG", IntegerType(), True),
    StructField("LIFT_ID", IntegerType(), True)
])

passengers_data = [
    ("Rahul", 85, 1),
    ("Adarsh", 73, 1),
    ("Riti", 95, 1),
    ("Dheeraj", 80, 1),
    ("Vimal", 83, 2),
    ("Neha", 77, 2),
    ("Priti", 73, 2),
    ("Himanshi", 85, 2)
]

df_passengers = spark.createDataFrame(passengers_data, schema=passengers_schema)
df_passengers.show()
df_passengers.createOrReplaceTempView("passengers")


+---+-----------+
| ID|CAPACITY_KG|
+---+-----------+
|  1|        300|
|  2|        350|
+---+-----------+

+--------------+---------+-------+
|PASSENGER_NAME|WEIGHT_KG|LIFT_ID|
+--------------+---------+-------+
|         Rahul|       85|      1|
|        Adarsh|       73|      1|
|          Riti|       95|      1|
|       Dheeraj|       80|      1|
|         Vimal|       83|      2|
|          Neha|       77|      2|
|         Priti|       73|      2|
|      Himanshi|       85|      2|
+--------------+---------+-------+



In [118]:
spark.sql("""
with cte as (select *, if(sum(WEIGHT_KG) over(partition by ID order by WEIGHT_KG) > CAPACITY_KG, 1, 0) as fl from passengers join lift on LIFT_ID = ID)
select LIFT_ID, concat_ws(" , ", collect_list(PASSENGER_NAME)) from cte where fl = 0 group by 1
""").show(truncate=False)

+-------+--------------------------------------------+
|LIFT_ID|concat_ws( , , collect_list(PASSENGER_NAME))|
+-------+--------------------------------------------+
|1      |Adarsh , Dheeraj , Rahul                    |
|2      |Priti , Neha , Vimal , Himanshi             |
+-------+--------------------------------------------+



In [5]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("TIME", StringType(), True),
    StructField("STATUS", StringType(), True)
])

data = [
    ("10:00:00", "on"),
    ("10:01:00", "on"),
    ("10:02:00", "on"),
    ("10:03:00", "off"),
    ("10:04:00", "on"),
    ("10:05:00", "on"),
    ("10:06:00", "off"),
    ("10:07:00", "off"),
    ("10:08:00", "off"),
    ("10:09:00", "on"),
    ("10:10:00", "on"),
    ("10:11:00", "on"),
    ("10:12:00", "on"),
    ("10:13:00", "off"),
    ("10:14:00", "off"),
    ("10:15:00", "on"),
    ("10:16:00", "off"),
    ("10:17:00", "off")
]
df = spark.createDataFrame(data, schema=schema)
df.show()
df.createOrReplaceTempView("df")

+--------+------+
|    TIME|STATUS|
+--------+------+
|10:00:00|    on|
|10:01:00|    on|
|10:02:00|    on|
|10:03:00|   off|
|10:04:00|    on|
|10:05:00|    on|
|10:06:00|   off|
|10:07:00|   off|
|10:08:00|   off|
|10:09:00|    on|
|10:10:00|    on|
|10:11:00|    on|
|10:12:00|    on|
|10:13:00|   off|
|10:14:00|   off|
|10:15:00|    on|
|10:16:00|   off|
|10:17:00|   off|
+--------+------+



In [15]:
spark.sql("""
with cte as (select *, lag(STATUS, 1, 'off') over(partition by 1 order by TIME) as fl from df),
cte2 as (select * from cte where STATUS != fl),
cte3 as (select *, row_number() over(partition by STATUS order by TIME) as rnk from cte2)
select min(TIME) logon, max(TIME) logoff, (unix_timestamp(max(TIME),'HH:mm:ss') - unix_timestamp(min(TIME),'HH:mm:ss') ) / 60 duration 
from cte3 group by rnk
""").show(truncate=False)

+--------+--------+--------+
|logon   |logoff  |duration|
+--------+--------+--------+
|10:00:00|10:03:00|3.0     |
|10:04:00|10:06:00|2.0     |
|10:09:00|10:13:00|4.0     |
|10:15:00|10:16:00|1.0     |
+--------+--------+--------+

